### IMEC2543 - DINAMICA DE MAQUINARIA
### [Luis Alejandro Rodríguez Arenas](https://luigitoby.github.io/) Cod. 202321287

In [1]:
import numpy as np
import sympy
from sympy import symbols, pi, solve
from sympy.physics.mechanics import ReferenceFrame, dynamicsymbols
import scipy
import matplotlib.pyplot as plt

# M1
## FK
* Entonces vemos como podemos dibujar el mecanismo haciendo un diccionario de puntos con luego tratamos este diccionario con list interpretation para volver estas funciones numericas para cada punto a funciones con el lambdify.
* Despues hacemos una funcción para graficar el mecanismo y ya tenemos todo lo necesario para poder simular el mecanismo.
* Hacemos la lista de posiciones y con lispace haceos una lista de tiempos, poscicion y theta
* Así podemos iterar cada lista hacemos el fsolve tipico
* Aquí la idea es que guardemos el out pero actualicemos la solución anterior como la siguiente initial guess.
* Porque el mecanismo no se teletransporta
* Resolvemos cada instant en el tiempo y busco todos los zetas, es decir los theta2 y theta3
* Ya con esto podemos realizar una simulación
* Note que en este código núnca hacemos una opción de detectar si no covnerge, esto es muy importante, aunque en el caso de la exp1 no lo es mucho
## IK
* En este código se necesita el FK, para poder garantizar que la solución también cierra el mecanismo.
* Eso cierra toda la parte anterior
* A esta se le pasa el q y me devuelve cual es el z y la convergencia
* Luego calcula la fucn de error de la IK, aprovechando las funciones del punto S definido arriba. Con el error de la posicion de S que es un error de distancia
* La idea es minimizar este error con una optimización
* Esto último se hace en un último bloque de IK que hace este minimize
* El minimize a veces para con escalas pequeñas por lo que es necesario multiplicar por 100 para aumentar el error manualmente. O bajarle la tolerancia al minimize.


In [ ]:
L1,  L2, L3, L4 = symbols('L1 L2 L3 L4')
theta1, theta2, theta3, theta4 = dynamicsymbols('theta1 theta2 theta3 theta4')

N = ReferenceFrame('N')
A = N.orientnew('A', 'Axis', [theta1, N.z])
B = N.orientnew('B', 'Axis', [theta2, N.z])
C = N.orientnew('C', 'Axis', [theta3-pi, N.z])

# Estos puntos deben seguir las trayectorías, a diferencia del robix, entonces debo hacer la cinemática inversa, pero no lo puedo llevar a cualquier punto
# entonces es un poco más complicado.

# Ecuación de cinemática

eqLoop = A.x*L1 +B.x*L2 + C.x*L3 - N.x*L4


eq = eqLoop.express(N)


val = {theta1:np.deg2rad(45)}

params = {L1:4, L2:3, L3:2, L4:5}

eqnum = eq.subs(val).subs(params)

eqnum

sln = sympy.solve([eqnum.dot(N.x),eqnum.dot(N.y)],[theta2, theta3])
sln_sym=np.rad2deg(np.array(sln[1],dtype=float))
print(sln_sym)
fun = sympy.lambdify([[theta2, theta3]], [eqnum.dot(N.x),eqnum.dot(N.y)]) 
# Me vuelve la lista de expresiones simbólicas una lsita de funciones numéricas

# np.rad2deg(np.array(sln[1],dtype=float))

# fun(np.deg2rad(30), np.deg2rad(150))

sln_num = scipy.optimize.fsolve(fun, [np.deg2rad(30), np.deg2rad(150)])
# sln_num = scipy.optimize.fsolve(fun, [np.deg2rad(30), np.deg2rad(150)], args=(), xtol=1.49012e-08, maxfev=0, full_output=False)

sln_num = np.rad2deg(sln_num)
print(sln_num)

# Esta es la solución positiva

[-18.37929778  70.26328001]
[-18.37929778  70.26328001]


In [4]:
theta1, theta2, theta3, ORx = dynamicsymbols('theta1 theta2 theta3 OR_x')
OP, PQ, QR, PS, ORy = symbols('OP PQ QR PS OR_y')

N = ReferenceFrame('N')
A = N.orientnew('A', 'Axis', [theta1, N.z])
B = N.orientnew('B', 'Axis', [theta2, N.z])
C = N.orientnew('C', 'Axis', [theta3, N.z])

# Posiciones

pos = OP*A.x + PQ*B.x + QR*C.x - ORx*N.x - ORy*N.y

eq = pos.express(N)

inicial = {theta1: np.deg2rad(67.5),
           ORx: 0.7}

params = {OP: 0.2,
          PQ: 0.15,
          QR: 0.6,
          PS: 0.15,
          ORy: 0.05}

eqnum = eq.subs(inicial).subs(params)

sol = solve([eqnum.dot(N.x), eqnum.dot(N.y)], [theta2, theta3])

sol = np.array(sol[1], dtype=float)

sol

array([ 0.98693191, -0.44805258])